In [ ]:
import json
import random
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from langchain.schema import HumanMessage

from Utils.Prompt import human_query_generation_agent

OPENAI_API_KEY = os.getenv('OPEN_AI_API')

def create_prompt():
    count = 0

    #day
    day_list = ['2 day','3 days', '4 days']
    day = random.choice(day_list)
    count += 1

    #price
    price_list = ["cheap budget","moderate budget","expensive budget"]
    price = random.choice(price_list)
    count += 1

    #attraction orientation
    attraction_orientation_list = ["family oriented","history oriented","activity oriented","nature oriented","food oriented","shopping oriented"]
    attraction_orientation = random.choice(attraction_orientation_list)
    count += 1

    #cuisine
    cuisine_list = ["US","Mexican","Irish","French","Italian","Greek","Indian","Chinese","Japanese","Korean","Vietnamese","Thai","Asian Fusion","Middle Eastern"]
    cuisine = random.choice(cuisine_list)
    count += 1

    #restaurant rating
    weights = [0.6, 0.3, 0.1]  # weights for 1, 2, 3, and 4 respectively
    samples = [1, 2, 3]
    coins = random.choices(samples, weights=weights)[0]
    n_list = ["good flavor","good freshness","good service","good environment","good value"]
    restaurant_n = random.sample(n_list, coins)
    restaurant_n_str = ', '.join(restaurant_n)
    count += coins
    
    #hotel rating
    weights = [0.6, 0.3, 0.1]  # weights for 1, 2, 3, and 4 respectively
    samples = [1, 2, 3]
    coins = random.choices(samples, weights=weights)[0]
    n_list = ["good quality","good location","good service","good safety"]
    hotel_n = random.sample(n_list, coins)
    hotel_n_str = ', '.join(hotel_n)
    count += coins

    #parse the prompt
    prompt = {
        "general":day + ", " + price,
        "attraction":attraction_orientation,
        "restaurant":cuisine + (', ' if len(restaurant_n) > 0 else "") + restaurant_n_str,
        "hotel":hotel_n_str
    }

    eval_info = {
        "day":[day],
        "price":[price],
        "attraction":[attraction_orientation],
        "cuisine":[cuisine],
        "restaurant":restaurant_n,
        "hotel":hotel_n,
        "preference_count":[count]
    }

    return prompt,eval_info

class QueryGenerationAgent():
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0,
                        model_name='gpt-4o-2024-11-20',
                        openai_api_key=OPENAI_API_KEY)
        
    def generateHumanQuery(self, input, generation_agent):
        content = generation_agent.format(input = input)
        request = self.llm.invoke([HumanMessage(content)]).content
        return request
    
if __name__ == "__main__":
    genAgent = QueryGenerationAgent()
    humanQuerys = []
    evals = []

    for i in range(20):
        prompt_input,eval_info = create_prompt()
        evals.append({"index":i+1,"eval_info":eval_info})
        humanQuery = genAgent.generateHumanQuery(prompt_input,human_query_generation_agent)
        humanQuerys.append({"index":i+1,"query":humanQuery})

    with open('Prompts/humanQuerys.jsonl','w') as file:
        for humanQuery in humanQuerys:
            json.dump(humanQuery, file)
            file.write('\n')

    with open('Prompts/evals.jsonl','w') as file:
        for eval in evals:
            json.dump(eval, file)
            file.write('\n')